# Requirments:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, RobustScaler 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor 
from sklearn.svm import SVR
from xgboost import XGBRFRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.datasets import make_classification
import warnings
warnings.filterwarnings("ignore")

# Input the data

In [ ]:
data = pd.read_csv("./data.csv")
feature_cols = ['FishWW', 'FishLW', 'FishEDI', 'MeatWW', 'MeatLW', 'MeatDEI','EggWW', 'EggLW', 'EggEDI','MilkWW', 'MilkLW', 'MilkEDI','LegumeWW', 'LegumeEDI', 'CerealWW','CerealEDI', 'VegetWW', 'VegetEDI','PotatoWW', 'PotatoEDI', 'SumEDI','Incineration', 'IOS', 'EAF', 'Nonferrous','HW1000', 'HW1001', 'HW1013','HW1031', 'HW1014', 'HW1051','HW1099', 'CloA40', 'CloT64','PheDP3', 'PheDP4', 'PheDP5', 'PheDP6', 'Sovol','Chlorofen', 'KC300', 'KC400','KC500', 'KC600', 'KC1016','KC1232', 'KC1248', 'KC1254','KC1260', 'KC1262', 'LogKow','LogKoa', 'LogBCF', 'WaSolu', 'LogVaPres','NBO', 'Viscosity', 'MP']
X = data[feature_cols]
y = data['HumanWW']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
knn = KNeighborsRegressor()
lin = LinearRegression()
dt = DecisionTreeRegressor()
rf = RandomForestRegressor()
bag = BaggingRegressor()
gbdt = GradientBoostingRegressor(loss='squared_error')
ada_dt = AdaBoostRegressor(base_estimator=dt)
ada_lin = AdaBoostRegressor(base_estimator=lin)
xg = XGBRFRegressor()
vote = VotingRegressor(estimators=[('rf_reg', rf_reg), ('ada_dt_reg',  ada_dt_reg)])
stack2_rf = RandomForestRegressor(n_estimators=500, oob_score=True)

model = [knn, lin, dt_reg, rf, gbdt, ada_dt, ridge, xg, svr]

In [ ]:
model_label = ['knn', 'lin', 'dt_reg', 'rf', 'gbdt', 'ada_dt', 'ridge', 'xg', 'svr']
 
 
assess_label = ['score', 'EVS', 'MSE', 'MAE','median', 'R2']
comparion = pd.DataFrame(index=model_label, columns=assess_label)

# RF

In [ ]:
def RF_evaluate(n_estimators, min_samples_split, max_features, max_depth, max_leaf_nodes):
    val = cross_val_score(
        RandomForestRegressor(n_estimators=int(n_estimators),
                              min_samples_split=int(min_samples_split),
                              max_features='auto',
                              max_depth=int(max_depth),
                              max_leaf_nodes=int(max_leaf_nodes),
                              random_state=1,
                              n_jobs=-1),
        X_train, y_train, scoring='r2', cv=10
        ).mean()  
    return val

pbounds = {'n_estimators': (10, 250),
           'min_samples_split': (2, 25),
           'max_features': (0.5, 10),
           'max_depth': (10, 50),
           'max_leaf_nodes': (50, 60)}        

RF_bo = BayesianOptimization(
        f=RF_evaluate,
        pbounds=pbounds,
        verbose=2,
        random_state=1)

RF_bo.maximize(init_points=5,
               n_iter=20,
               acq='ei')

print(RF_bo.max)
res = RF_bo.max
params_max = res['params']

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))
y_pre = rf.predict(X_test)
score = rf.score(X_test, y_test)
EVS = explained_variance_score(y_test, y_pre)
MSE = mean_squared_error(y_test, y_pre)
MAE = mean_absolute_error(y_test, y_pre)
MSLE = mean_squared_log_error(y_test, y_pre)
median = median_absolute_error(y_test, y_pre)
r2 = r2_score(y_test, y_pre)
y_pre_train = rf.predict(X_train)

print('assess_label')

# SHAP

In [ ]:
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.interpolation'] = 'none'

explainer = shap.Explainer(rf, X)
shap_values = explainer(X_test)

shap.summary_plot(shap_values, X_test)
shap.summary_plot(shap_values, X_test, plot_type="bar")
shap.plots.bar(shap_values, max_display=None)

shap.plots.waterfall(shap_values[5], max_display=10)
shap.plots.waterfall(shap_values[0])
shap.plots.waterfall(shap_values[12])
shap.plots.waterfall(shap_values[32])
shap.plots.waterfall(shap_values[10])
shap.plots.force(shap_values[110])
shap.force_plot(explainer.expected_value, shap_values.values, X)

# XGBoost

In [ ]:
def Xgboost_evaluate(n_estimators, min_child_weight):
    val = cross_val_score(
            XGBRFRegressor(n_estimators=int(n_estimators),
                           min_child_weight=int(min_child_weight)),
            X_train, y_train, scoring='r2', cv=10
        ).mean()
    return val

pbounds = {'n_estimators': (50, 300),
           'min_child_weight': (2,10)}

Xgboost_bo = BayesianOptimization(f=Xgboost_evaluate,
                                  pbounds=pbounds,
                                  verbose=2,
                                  random_state=1)

Xgboost_bo.maximize(init_points=5,
                    n_iter=20,
                    acq='ei')

print(Xgboost_bo.max)
res = Xgboost_bo.max
params_max = res['params']

xg = XGBRFRegressor()
xg.fit(X_train, y_train)
print(xg.score(X_test, y_test))
y_pre = xg.predict(X_test)
score = xg.score(X_test, y_test)
EVS = explained_variance_score(y_test, y_pre)
MSE = mean_squared_error(y_test, y_pre)
MAE = mean_absolute_error(y_test, y_pre)
median = median_absolute_error(y_test, y_pre)
r2 = r2_score(y_test, y_pre)
y_pre_train = xg.predict(X_train)

print('assess_label')

# Linear

In [ ]:
lin = LinearRegression()
lin.fit(X_train, y_train)
print(lin.score(X_test, y_test))
y_pre = lin.predict(X_test)
score = lin.score(X_test, y_test)
EVS = explained_variance_score(y_test, y_pre)
MSE = mean_squared_error(y_test, y_pre)
MAE = mean_absolute_error(y_test, y_pre)
median = median_absolute_error(y_test, y_pre)
r2 = r2_score(y_test, y_pre)
y_pre_train = lin.predict(X_train)

print('assess_label')

# KNN

In [ ]:
def knn_evaluate(n_neighbors, leaf_size):
    val = cross_val_score(
            KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                leaf_size=int(leaf_size)),
            X_train, y_train, scoring='r2', cv=10
        ).mean()    
    return val

pbounds = {'n_neighbors': (2, 10),'leaf_size': (50,250)}          

knn_bo = BayesianOptimization(f=knn_evaluate,
                              pbounds=pbounds,
                              verbose=2,
                              random_state=1)

knn_bo.maximize(init_points=5,
                n_iter=20,
                acq='ei')

print(knn_bo.max)
res = knn_bo.max
params_max = res['params']

knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))
y_pre = knn.predict(X_test)
score = knn.score(X_test, y_test)
EVS = explained_variance_score(y_test, y_pre)
MSE = mean_squared_error(y_test, y_pre)
MAE = mean_absolute_error(y_test, y_pre)
median = median_absolute_error(y_test, y_pre)
r2 = r2_score(y_test, y_pre)
y_pre_train = knn.predict(X_train)

print('assess_label')

# Decision Tree

In [ ]:
def dt_evaluate(min_samples_split, max_features, max_depth, max_leaf_nodes):
    val = cross_val_score(
        DecisionTreeRegressor(min_samples_split=int(min_samples_split),
                              max_features='auto',
                              max_depth=int(max_depth),
                              max_leaf_nodes=int(max_leaf_nodes),
                              random_state=1),
        X_train, y_train, scoring='r2', cv=10).mean()    
    return val

pbounds = {'min_samples_split': (2, 25),
           'max_features': (0.5, 10),
           'max_depth': (10, 50),
           'max_leaf_nodes': (50, 60)}          

dt_bo = BayesianOptimization(f=dt_evaluate,
                             pbounds=pbounds,
                             verbose=2,
                             random_state=1)

dt_bo.maximize(init_points=5,
               n_iter=20,
               acq='ei')

print(dt_bo.max)
res = dt_bo.max
params_max = res['params']

dt_reg = DecisionTreeRegressor()
dt_reg.fit(X_train, y_train)
print(dt_reg.score(X_test, y_test))
y_pre = dt_reg.predict(X_test)
score = dt_reg.score(X_test, y_test)
EVS = explained_variance_score(y_test, y_pre)
MSE = mean_squared_error(y_test, y_pre)
MAE = mean_absolute_error(y_test, y_pre)
median = median_absolute_error(y_test, y_pre)
r2 = r2_score(y_test, y_pre)
y_pre_train = dt_reg.predict(X_train)

print('assess_label')

# GDBT

In [ ]:
def gbdt_evaluate(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_leaf_nodes, max_features):
    val = cross_val_score(
            GradientBoostingRegressor(n_estimators=int(n_estimators),
                                      min_samples_split=int(min_samples_split),
                                      max_depth=int(max_depth),
                                      max_leaf_nodes=int(max_leaf_nodes),
                                      min_samples_leaf=int(min_samples_leaf),
                                      max_features=int(max_features)),
            X_train, y_train, scoring='r2', cv=10
        ).mean()
    return val

pbounds = {'n_estimators': (20, 81),
           'min_samples_split': (100, 801),
           'max_features': (7, 20),
           'max_depth': (3, 14),
           'min_samples_leaf': (60, 101),
           'max_leaf_nodes': (50, 61)}         

gbdt_bo = BayesianOptimization(f=gbdt_evaluate,
                               pbounds=pbounds,
                               verbose=2,
                               random_state=1)

gbdt_bo.maximize(init_points=5,
                 n_iter=20,
                 acq='ei')

print(gbdt_bo.max)
res = gbdt_bo.max
params_max = res['params']

gbdt = GradientBoostingRegressor()
gbdt.fit(X_train, y_train)
print(gbdt.score(X_test, y_test))
y_pre = gbdt.predict(X_test)
score = gbdt.score(X_test, y_test)
EVS = explained_variance_score(y_test, y_pre)
MSE = mean_squared_error(y_test, y_pre)
MAE = mean_absolute_error(y_test, y_pre)
median = median_absolute_error(y_test, y_pre)
r2 = r2_score(y_test, y_pre)
y_pre_train = gbdt.predict(X_train)

print('assess_label')

# AdaBoost

In [ ]:
def ada_evaluate(n_estimators, learning_rate):
    val = cross_val_score(
            AdaBoostRegressor(n_estimators=int(n_estimators),
                              learning_rate=1.0),
            X_train, y_train, scoring='r2', cv=10
        ).mean()
    return val
    
pbounds = {'n_estimators': (50, 301),
           'learning_rate': (0, 1)}

ada_bo = BayesianOptimization(f=ada_evaluate,
                              pbounds=pbounds,
                              verbose=2,
                              random_state=1)

ada_bo.maximize(init_points=5,
                n_iter=20,
                acq='ei')

print(ada_bo.max)
res = ada_bo.max
params_max = res['params']

ada_bo = AdaBoostRegressor()
ada_bo.fit(X_train, y_train)
print(ada_bo.score(X_test, y_test))
y_pre = ada_bo.predict(X_test)
score = ada_bo.score(X_test, y_test)
EVS = explained_variance_score(y_test, y_pre)
MSE = mean_squared_error(y_test, y_pre)
MAE = mean_absolute_error(y_test, y_pre)
median = median_absolute_error(y_test, y_pre)
r2 = r2_score(y_test, y_pre)
y_pre_train = ada_bo.predict(X_train)

print('assess_label')

# Ridge

In [ ]:
def ridge_evaluate(alpha, tol):
    val = cross_val_score(
            Ridge(alpha=int(alpha),
                  tol=1),
            X_train, y_train, scoring='r2', cv=10
        ).mean()
    return val

pbounds = {'alpha': (1, 8),
           'tol': (0.00001, 1)}

ridge_bo = BayesianOptimization(f=ridge_evaluate,
                                pbounds=pbounds,
                                verbose=2,
                                random_state=1)

ridge_bo.maximize(init_points=5,
                  n_iter=20,
                  acq='ei')

print(ridge_bo.max)
res = ridge_bo.max
params_max = res['params']

ridge=Ridge()
ridge.fit(X_train, y_train)
print(ridge.score(X_test, y_test))
y_pre = ridge.predict(X_test)
score = ridge.score(X_test, y_test)
EVS = explained_variance_score(y_test, y_pre)
MSE = mean_squared_error(y_test, y_pre)
MAE = mean_absolute_error(y_test, y_pre)
median = median_absolute_error(y_test, y_pre)
r2 = r2_score(y_test, y_pre)
y_pre_train = ridge.predict(X_train)

print('assess_label')

# SVM

In [ ]:
def svr_evaluate(gamma, C):
    val = cross_val_score(
        SVR(gamma=1,
            C=1),
        X_train, y_train, scoring='r2', cv=10
        ).mean()
    return val

pbounds = {'gamma': (0.001, 10), 
           'C': (0.0001, 10)}

svr_bo = BayesianOptimization(f=svr_evaluate, 
                              pbounds=pbounds,
                              verbose=2, 
                              random_state=1)

svr_bo.maximize(init_points=5,
                n_iter=20, 
                acq='ei')

print(svr_bo.max)
res = svr_bo.max
params_max = res['params']

svr=SVR()
svr.fit(X_train, y_train)
print(svr.score(X_test, y_test))
y_pre = svr.predict(X_test)
score = svr.score(X_test, y_test)
EVS = explained_variance_score(y_test, y_pre)
MSE = mean_squared_error(y_test, y_pre)
MAE = mean_absolute_error(y_test, y_pre)
median = median_absolute_error(y_test, y_pre)
r2 = r2_score(y_test, y_pre)
y_pre_train = svr.predict(X_train)

print('assess_label')